 # Capstone Project - The Battle of Neighborhoods

## Import Libraries

In this section we import the libraries that will be required to process the data.

In [145]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

!pip install geopy
!pip install wget
from geopy.geocoders import Nominatim
import urllib.request
import json
from bs4 import BeautifulSoup
from urllib.request import urlopen
import requests
from pandas.io.json import json_normalize

import matplotlib.cm as cm
import matplotlib.colors as colors
import matplotlib.cm as cm
import matplotlib.pyplot as plt
import matplotlib.colors as colors
%matplotlib inline
from sklearn.cluster import KMeans

import folium


## Download and Explore Dataset


Download and Explore Dataset
Neighborhood has a total of 5 boroughs and 306 neighborhoods. In order to segement the neighborhoods and explore them, we will essentially need a dataset that contains the 5 boroughs and the neighborhoods that exist in each borough as well as the the latitude and logitude coordinates of each neighborhood.

The link to the dataset: https://cocl.us/new_york_dataset

In [146]:
!wget -q -O 'newyork_data.json' https://cocl.us/new_york_dataset
print('Data downloaded!')

with open('newyork_data.json') as json_data:
    newyork_data = json.load(json_data)

Data downloaded!


'wget' is not recognized as an internal or external command,
operable program or batch file.


#### Tranform the data into a *pandas* dataframe

In [147]:
neighborhoods_data = newyork_data['features']

column_names = ['Borough', 'Neighborhood', 'Latitude', 'Longitude'] 

neighborhoods = pd.DataFrame(columns=column_names)

for data in neighborhoods_data:
    borough = neighborhood_name = data['properties']['borough'] 
    neighborhood_name = data['properties']['name']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[1]
    neighborhood_lon = neighborhood_latlon[0]
    
    neighborhoods = neighborhoods.append({'Borough': borough,
                                          'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon}, ignore_index=True)

In [148]:
neighborhoods.head()

,Borough,Neighborhood,Latitude,Longitude
0,Bronx,Wakefield,40.894705,-73.847201
1,Bronx,Co-op City,40.874294,-73.829939
2,Bronx,Eastchester,40.887556,-73.827806
3,Bronx,Fieldston,40.895437,-73.905643
4,Bronx,Riverdale,40.890834,-73.912585


#### Use of geopy library to get the latitude and longitude values of New York City.

In [149]:
address = 'New York City, NY'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of New York City are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of New York City are 40.7127281, -74.0060152.


#### Map of New York with neighborhoods superimposed on top.

In [150]:
manhattan_data = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
manhattan_data.head()

,Borough,Neighborhood,Latitude,Longitude
0,Manhattan,Marble Hill,40.876551,-73.910660
1,Manhattan,Chinatown,40.715618,-73.994279
2,Manhattan,Washington Heights,40.851903,-73.936900
3,Manhattan,Inwood,40.867684,-73.921210
4,Manhattan,Hamilton Heights,40.823604,-73.949688


In [151]:
import folium
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=11)

for lat, lng, borough, neighborhood in zip(manhattan_data['Latitude'], manhattan_data['Longitude'], manhattan_data['Borough'], manhattan_data['Neighborhood']):
    label = '{}, {}'.format(neighborhood, borough)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.5,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

## Foursquare venues


In [152]:
import urllib
def getNearbyVenues(names, latitudes, longitudes, radius=5000, categoryIds=''):
    try:
        venues_list=[]
        for name, lat, lng in zip(names, latitudes, longitudes):

            url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(CLIENT_ID, CLIENT_SECRET, VERSION, lat, lng, radius, LIMIT)

            if (categoryIds != ''):
                url = url + '&categoryId={}'
                url = url.format(categoryIds)

            response = requests.get(url).json()
            results = response["response"]['venues']

            for v in results:
                success = False
                try:
                    category = v['categories'][0]['name']
                    success = True
                except:
                    pass

                if success:
                    venues_list.append([(
                        name, 
                        lat, 
                        lng, 
                        v['name'], 
                        v['location']['lat'], 
                        v['location']['lng'],
                        v['categories'][0]['name']
                    )])

        nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
        nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude',  
                  'Venue Category']
    
    except:
        print(url)
        print(response)
        print(results)
        print(nearby_venues)

    return(nearby_venues)

In [153]:
LIMIT = 500 
radius = 5000 
CLIENT_ID = '13AZBAZPH2RPYU0AHBRPS0SPZGQFJNP5UVCXFBAG2SO1XFVE'
CLIENT_SECRET = '1B3QNU1V2OHCKLCIKTPCPKS4DIODB3OMUBQMH3Q2ACDHE04W'
VERSION = '20181020'

In [154]:
neighborhoods = neighborhoods[neighborhoods['Borough'] == 'Manhattan'].reset_index(drop=True)
newyork_venues_thai = getNearbyVenues(names=neighborhoods['Neighborhood'], latitudes=neighborhoods['Latitude'], longitudes=neighborhoods['Longitude'], radius=1000, categoryIds='4bf58dd8d48988d149941735')
newyork_venues_thai.head()

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,Marble Hill,40.876551,-73.910660,Siam Square,40.878796,-73.916701,Thai Restaurant
1,Marble Hill,40.876551,-73.910660,Nam Thai,40.886388,-73.910025,Thai Restaurant
2,Chinatown,40.715618,-73.994279,Noree Thai Bazaar,40.717900,-73.992966,Thai Restaurant
3,Chinatown,40.715618,-73.994279,Thai Diner,40.720739,-73.995631,Thai Restaurant
4,Chinatown,40.715618,-73.994279,Wayla,40.718291,-73.992584,Thai Restaurant


In [155]:
newyork_venues_thai.shape

(1014, 7)

In [156]:
def addToMap(df, color, existingMap):
    for lat, lng, local, venue, venueCat in zip(df['Venue Latitude'], df['Venue Longitude'], df['Neighborhood'], df['Venue'], df['Venue Category']):
        label = '{} ({}) - {}'.format(venue, venueCat, local)
        label = folium.Popup(label, parse_html=True)
        folium.CircleMarker(
            [lat, lng],
            radius=5,
            popup=label,
            color=color,
            fill=True,
            fill_color=color,
            fill_opacity=0.7).add_to(existingMap)

In [157]:
map_newyork_thai = folium.Map(location=[latitude, longitude], zoom_start=10)
addToMap(newyork_venues_thai, 'red', map_newyork_thai)

map_newyork_thai

In [158]:
def addColumn(startDf, columnTitle, dataDf):
    grouped = dataDf.groupby('Neighborhood').count()
    
    for n in startDf['Neighborhood']:
        try:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = grouped.loc[n, 'Venue']
        except:
            startDf.loc[startDf['Neighborhood'] == n,columnTitle] = 0

In [159]:
manhattan_grouped = newyork_venues_thai.groupby('Neighborhood').count()
print(manhattan_grouped)

print('There are {} uniques categories.'.format(len(newyork_venues_thai['Venue Category'].unique())))

                     Neighborhood Latitude  Neighborhood Longitude  Venue  \
Neighborhood                                                                
Battery Park City                       10                      10     10   
Carnegie Hill                           25                      25     25   
Central Harlem                           6                       6      6   
Chelsea                                 31                      31     31   
Chinatown                               36                      36     36   
Civic Center                            23                      23     23   
Clinton                                 48                      48     48   
East Harlem                             10                      10     10   
East Village                            49                      49     49   
Financial District                      11                      11     11   
Flatiron                                46                      46     46   

## 3. Analyze Each Neighborhood

In [160]:
# one hot encoding
manhattan_onehot = pd.get_dummies(newyork_venues_thai[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
manhattan_onehot['Neighborhood'] = newyork_venues_thai['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [manhattan_onehot.columns[-1]] + list(manhattan_onehot.columns[:-1])
manhattan_onehot = manhattan_onehot[fixed_columns]

manhattan_onehot.head()

,Neighborhood,Asian Restaurant,Chinese Restaurant,Filipino Restaurant,Food Truck,Indian Restaurant,Japanese Restaurant,Malay Restaurant,Ramen Restaurant,Sushi Restaurant,Thai Restaurant,Vietnamese Restaurant,Wine Bar
0,Marble Hill,0,0,0,0,0,0,0,0,0,1,0,0
1,Marble Hill,0,0,0,0,0,0,0,0,0,1,0,0
2,Chinatown,0,0,0,0,0,0,0,0,0,1,0,0
3,Chinatown,0,0,0,0,0,0,0,0,0,1,0,0
4,Chinatown,0,0,0,0,0,0,0,0,0,1,0,0


In [161]:
manhattan_grouped = manhattan_onehot.groupby('Neighborhood').mean().reset_index()
manhattan_grouped

,Neighborhood,Asian Restaurant,Chinese Restaurant,Filipino Restaurant,Food Truck,Indian Restaurant,Japanese Restaurant,Malay Restaurant,Ramen Restaurant,Sushi Restaurant,Thai Restaurant,Vietnamese Restaurant,Wine Bar
0,Battery Park City,0.000000,0.000000,0.000000,0.100000,0.000000,0.100000,0.000000,0.000000,0.000000,0.800000,0.000000,0.000000
1,Carnegie Hill,0.040000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.960000,0.000000,0.000000
2,Central Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
3,Chelsea,0.000000,0.032258,0.000000,0.032258,0.000000,0.032258,0.032258,0.000000,0.000000,0.870968,0.000000,0.000000
4,Chinatown,0.083333,0.000000,0.027778,0.000000,0.000000,0.000000,0.027778,0.000000,0.027778,0.805556,0.027778,0.000000
5,Civic Center,0.043478,0.000000,0.000000,0.043478,0.000000,0.043478,0.043478,0.000000,0.000000,0.826087,0.000000,0.000000
6,Clinton,0.020833,0.020833,0.000000,0.041667,0.000000,0.000000,0.000000,0.041667,0.000000,0.875000,0.000000,0.000000
7,East Harlem,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000
8,East Village,0.040816,0.040816,0.020408,0.020408,0.000000,0.000000,0.000000,0.000000,0.040816,0.795918,0.020408,0.020408
9,Financial District,0.000000,0.000000,0.000000,0.090909,0.000000,0.090909,0.000000,0.000000,0.000000,0.818182,0.000000,0.000000


In [162]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
    row_categories_sorted = row_categories.sort_values(ascending=False)
    
    return row_categories_sorted.index.values[0:num_top_venues]

In [163]:
num_top_venues = 10

indicators = ['st', 'nd', 'rd']

columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = manhattan_grouped['Neighborhood']

for ind in np.arange(manhattan_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(manhattan_grouped.iloc[ind, :], num_top_venues)

neighborhoods_venues_sorted.head()

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Battery Park City,Thai Restaurant,Japanese Restaurant,Food Truck,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Indian Restaurant,Filipino Restaurant
1,Carnegie Hill,Thai Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
2,Central Harlem,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
3,Chelsea,Thai Restaurant,Malay Restaurant,Japanese Restaurant,Food Truck,Chinese Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Indian Restaurant
4,Chinatown,Thai Restaurant,Asian Restaurant,Vietnamese Restaurant,Sushi Restaurant,Malay Restaurant,Filipino Restaurant,Wine Bar,Ramen Restaurant,Japanese Restaurant,Indian Restaurant


Cluster Neighborhoods


In [164]:
kclusters = 5

manhattan_grouped_clustering = manhattan_grouped.drop('Neighborhood', 1)

kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(manhattan_grouped_clustering)

kmeans.labels_[0:10] 

array([3, 4, 1, 0, 2, 3, 0, 1, 2, 3])

In [165]:
neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)

manhattan_merged = manhattan_data
manhattan_merged = manhattan_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighborhood')

manhattan_merged.head() 
manhattan_merged.dropna(axis=1)

,Borough,Neighborhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Manhattan,Marble Hill,40.876551,-73.910660,1,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
1,Manhattan,Chinatown,40.715618,-73.994279,2,Thai Restaurant,Asian Restaurant,Vietnamese Restaurant,Sushi Restaurant,Malay Restaurant,Filipino Restaurant,Wine Bar,Ramen Restaurant,Japanese Restaurant,Indian Restaurant
2,Manhattan,Washington Heights,40.851903,-73.936900,1,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
3,Manhattan,Inwood,40.867684,-73.921210,1,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
4,Manhattan,Hamilton Heights,40.823604,-73.949688,1,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
5,Manhattan,Manhattanville,40.816934,-73.957385,1,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
6,Manhattan,Central Harlem,40.815976,-73.943211,1,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
7,Manhattan,East Harlem,40.792249,-73.944182,1,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
8,Manhattan,Upper East Side,40.775639,-73.960508,4,Thai Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
9,Manhattan,Yorkville,40.775930,-73.947118,4,Thai Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck


In [166]:
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

markers_colors = []
for lat, lon, poi, cluster in zip(manhattan_merged['Latitude'], manhattan_merged['Longitude'], manhattan_merged['Neighborhood'], manhattan_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [167]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 0, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]

,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
13,Lincoln Square,Thai Restaurant,Ramen Restaurant,Food Truck,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant
14,Clinton,Thai Restaurant,Ramen Restaurant,Food Truck,Chinese Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Malay Restaurant,Japanese Restaurant
15,Midtown,Thai Restaurant,Food Truck,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant
17,Chelsea,Thai Restaurant,Malay Restaurant,Japanese Restaurant,Food Truck,Chinese Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Indian Restaurant
27,Gramercy,Thai Restaurant,Food Truck,Chinese Restaurant,Malay Restaurant,Japanese Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant
33,Midtown South,Thai Restaurant,Food Truck,Japanese Restaurant,Chinese Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Indian Restaurant
36,Tudor City,Thai Restaurant,Food Truck,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant
38,Flatiron,Thai Restaurant,Food Truck,Chinese Restaurant,Malay Restaurant,Japanese Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant


In [168]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 1, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,Marble Hill,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
2,Washington Heights,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
3,Inwood,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
4,Hamilton Heights,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
5,Manhattanville,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
6,Central Harlem,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
7,East Harlem,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
10,Lenox Hill,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
11,Roosevelt Island,Thai Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck,Filipino Restaurant
24,West Village,Thai Restaurant,Chinese Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck


In [169]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 2, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
1,Chinatown,Thai Restaurant,Asian Restaurant,Vietnamese Restaurant,Sushi Restaurant,Malay Restaurant,Filipino Restaurant,Wine Bar,Ramen Restaurant,Japanese Restaurant,Indian Restaurant
19,East Village,Thai Restaurant,Sushi Restaurant,Chinese Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Food Truck,Filipino Restaurant,Ramen Restaurant,Malay Restaurant
20,Lower East Side,Thai Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Filipino Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant
22,Little Italy,Thai Restaurant,Asian Restaurant,Vietnamese Restaurant,Sushi Restaurant,Malay Restaurant,Filipino Restaurant,Wine Bar,Ramen Restaurant,Japanese Restaurant,Indian Restaurant
23,Soho,Thai Restaurant,Asian Restaurant,Sushi Restaurant,Malay Restaurant,Wine Bar,Vietnamese Restaurant,Ramen Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
25,Manhattan Valley,Thai Restaurant,Indian Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Food Truck
31,Noho,Thai Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Filipino Restaurant,Chinese Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant


In [170]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 3, manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
16,Murray Hill,Thai Restaurant,Food Truck,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant
28,Battery Park City,Thai Restaurant,Japanese Restaurant,Food Truck,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Indian Restaurant,Filipino Restaurant
29,Financial District,Thai Restaurant,Japanese Restaurant,Food Truck,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Indian Restaurant,Filipino Restaurant
32,Civic Center,Thai Restaurant,Malay Restaurant,Japanese Restaurant,Food Truck,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Indian Restaurant
34,Sutton Place,Thai Restaurant,Food Truck,Chinese Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant
35,Turtle Bay,Thai Restaurant,Food Truck,Asian Restaurant,Chinese Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant


In [172]:
manhattan_merged.loc[manhattan_merged['Cluster Labels'] == 4 , manhattan_merged.columns[[1] + list(range(5, manhattan_merged.shape[1]))]]


,Neighborhood,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Upper East Side,Thai Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
9,Yorkville,Thai Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
12,Upper West Side,Thai Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
18,Greenwich Village,Thai Restaurant,Asian Restaurant,Sushi Restaurant,Chinese Restaurant,Wine Bar,Vietnamese Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant
21,Tribeca,Thai Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
26,Morningside Heights,Thai Restaurant,Indian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Food Truck,Filipino Restaurant
30,Carnegie Hill,Thai Restaurant,Asian Restaurant,Wine Bar,Vietnamese Restaurant,Sushi Restaurant,Ramen Restaurant,Malay Restaurant,Japanese Restaurant,Indian Restaurant,Food Truck
